<a href="https://colab.research.google.com/github/ayushjain1144/semantic-segmentation-IGCAR/blob/master/bert_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/google-research/bert

Cloning into 'bert'...
remote: Enumerating objects: 329, done.
remote: Total 329 (delta 0), reused 0 (delta 0), pack-reused 329
Receiving objects: 100% (329/329), 239.74 KiB | 876.00 KiB/s, done.
Resolving deltas: 100% (188/188), done.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/igcar_ps/ /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf
#import sentencepiece as spm

from glob import glob
from google.colab import auth, drive
from tensorflow.keras.utils import Progbar

sys.path.append("bert")



from bert import modeling, optimization, tokenization
#from bert.run_pretraining import input_fn_builder, model_fn_builder

auth.authenticate_user()

log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s : \
    %(message)s')
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
log.handlers = [sh]

if 'COLAB_TPU_ADDR' in os.environ:
    log.info("Using TPU runtime")
    USE_TPU = True
    TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    
    with tf.Session(TPU_ADDRESS) as session:
        log.info('TPU address is ' + TPU_ADDRESS)
        with open('/content/adc.json', 'r') as f:
          auth_info = json.load(f)
        
        tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
        
       
else:
    log.warning('Not connected to TPU runtime')
    USE_TPU = False

W0702 03:48:17.670795 139977353672576 deprecation_wrapper.py:119] From /content/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

2019-07-02 03:48:51,638 :     Not connected to TPU runtime
W0702 03:48:51.638993 139977353672576 interactiveshell.py:2882] Not connected to TPU runtime


In [4]:
BUCKET_PATH = "gs://ayushjain1144-bucket/"
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = BUCKET_PATH + BERT_MODEL
print('****** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR)) 
#!ls BERT_PRETRAINED_DIR

****** BERT pretrained directory: gs://ayushjain1144-bucket/uncased_L-12_H-768_A-12 *****


In [0]:
BERT_CONFIG = BERT_PRETRAINED_DIR + '/bert_config.json'
CHKPT_DIR = BERT_PRETRAINED_DIR + '/bert_model.ckpt.*'
VOCAB_FILE = BERT_PRETRAINED_DIR + '/vocab.txt'
INIT_CHECKPOINT = BERT_PRETRAINED_DIR + '/bert_model.ckpt'

In [0]:
QA_DATASET = BUCKET_PATH + 'qa_dataset'
QA_TRAIN_FILE = QA_DATASET + '/train-v1.1.json'
QA_PREDICT_FILE = QA_DATASET + '/dev-v1.1.json'
QA_EVALUATE = QA_DATASET + '/evaluate-v1.1.py'


In [0]:
OUTPUT_DIR = BUCKET_PATH + 'output-dir/'

In [0]:
PRED_JSON = OUTPUT_DIR + '/predictions.json'

2019-07-01 19:00:08,584 :     
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0701 19:00:08.584650 139813024868224 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



NameError: ignored

SyntaxError: ignored

In [9]:
XARGS_CMD = ("python3 bert/run_squad.py "
            "--vocab_file={} "
            "--bert_config_file={} "
            "--init_checkpoint={} "
            "--do_train=True "
            "--train_file={} "
            "--do_predict=True "
            "--predict_file={} "
            "--train_batch_size=12 "
            "--learning_rate=3e-5 "
            "--num_train_epochs=2.0 "
            "--max_seq_length=256 "
            "--doc_stride=128 "
            "--output_dir={} "
            )
XARGS_CMD = XARGS_CMD.format(VOCAB_FILE, 
                            BERT_CONFIG,
                            INIT_CHECKPOINT,
                            QA_TRAIN_FILE,
                            QA_PREDICT_FILE,
                            OUTPUT_DIR)
                           
print(XARGS_CMD)

python3 bert/run_squad.py --vocab_file=gs://ayushjain1144-bucket/uncased_L-12_H-768_A-12/vocab.txt --bert_config_file=gs://ayushjain1144-bucket/uncased_L-12_H-768_A-12/bert_config.json --init_checkpoint=gs://ayushjain1144-bucket/uncased_L-12_H-768_A-12/bert_model.ckpt --do_train=True --train_file=gs://ayushjain1144-bucket/qa_dataset/train-v1.1.json --do_predict=True --predict_file=gs://ayushjain1144-bucket/qa_dataset/dev-v1.1.json --train_batch_size=12 --learning_rate=3e-5 --num_train_epochs=2.0 --max_seq_length=256 --doc_stride=128 --output_dir=gs://ayushjain1144-bucket/output-dir/ 


In [0]:
!$XARGS_CMD

In [12]:
!python3 /mydrive/qa_dataset/evaluate-v1.1.py /mydrive/qa_dataset/dev-v1.1.json output-dir_predictions.json

{"exact_match": 80.54872280037843, "f1": 88.10709530798852}


In [13]:
!cat > results.txt

model: BERT + SQUAD
max_seq_length:256 
train_batch_size: 12
exact match: 80.5487
f1: 88.107


KeyboardInterrupt: ignored